Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/training/train-on-amlcompute/train-on-amlcompute.png)

#  Train using Azure Machine Learning Compute

* Initialize a Workspace
* Create an Experiment
* Introduction to AmlCompute
* Submit an AmlCompute run in a few different ways
    - Provision as a run based compute target 
    - Provision as a persistent compute target (Basic)
    - Provision as a persistent compute target (Advanced)
* Additional operations to perform on AmlCompute
* Find the best model in the run

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set.  Otherwise, go through the [configuration](../../../configuration.ipynb) Notebook first if you haven't already to establish your connection to the AzureML Workspace.

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.85


## Initialize a Workspace

Initialize a workspace object from persisted configuration

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

cirrustest2
awe-cirrus-rg
southcentralus
c1d0a0ea-bf6e-4c1c-8b55-f1bdb0208df8


## Create An Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [3]:
from azureml.core import Experiment
experiment_name = 'train-on-amlcompute-01'
experiment = Experiment(workspace = ws, name = experiment_name)

## Introduction to AmlCompute

**Azure Machine Learning Compute is managed compute infrastructure that allows the user to easily create single to multi-node compute of the appropriate VM Family**. It is created **within your workspace region** and is a resource that can be used by other users in your workspace. It autoscales by default to the max_nodes, when a job is submitted, and executes in a containerized environment packaging the dependencies as specified by the user. 

Since it is managed compute, job scheduling and cluster management are handled internally by Azure Machine Learning service. 

For more information on Azure Machine Learning Compute, please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)

If you are an existing BatchAI customer who is migrating to Azure Machine Learning, please read [this article](https://aka.ms/batchai-retirement)

**Note**: As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.


The training script `train.py` is already created for you. Let's have a look.

## Submit an AmlCompute run in a few different ways

First lets check which VM families are available in your region. Azure is a regional service and some specialized SKUs (especially GPUs) are only available in certain regions. Since AmlCompute is created in the region of your workspace, we will use the supported_vms () function to see if the VM family we want to use ('STANDARD_D2_V2') is supported.

You can also pass a different region to check availability and then re-create your workspace in that region through the [configuration notebook](../../../configuration.ipynb)

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

AmlCompute.supported_vmsizes(workspace = ws,location='southcentralus')
#AmlCompute.supported_vmsizes(workspace = ws, location='southcentralus')

[{'name': 'Standard_D1_v2',
  'vCPUs': 1,
  'gpus': 0,
  'memoryGB': 3.5,
  'maxResourceVolumeMB': 51200},
 {'name': 'Standard_D2_v2',
  'vCPUs': 2,
  'gpus': 0,
  'memoryGB': 7.0,
  'maxResourceVolumeMB': 102400},
 {'name': 'Standard_D3_v2',
  'vCPUs': 4,
  'gpus': 0,
  'memoryGB': 14.0,
  'maxResourceVolumeMB': 204800},
 {'name': 'Standard_D4_v2',
  'vCPUs': 8,
  'gpus': 0,
  'memoryGB': 28.0,
  'maxResourceVolumeMB': 409600},
 {'name': 'Standard_D11_v2',
  'vCPUs': 2,
  'gpus': 0,
  'memoryGB': 14.0,
  'maxResourceVolumeMB': 102400},
 {'name': 'Standard_D12_v2',
  'vCPUs': 4,
  'gpus': 0,
  'memoryGB': 28.0,
  'maxResourceVolumeMB': 204800},
 {'name': 'Standard_D13_v2',
  'vCPUs': 8,
  'gpus': 0,
  'memoryGB': 56.0,
  'maxResourceVolumeMB': 409600},
 {'name': 'Standard_D14_v2',
  'vCPUs': 16,
  'gpus': 0,
  'memoryGB': 112.0,
  'maxResourceVolumeMB': 819200},
 {'name': 'Standard_DS1_v2',
  'vCPUs': 1,
  'gpus': 0,
  'memoryGB': 3.5,
  'maxResourceVolumeMB': 7168},
 {'name': 'Standar

### Create project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on

In [5]:
import os
import shutil

project_folder = './train-on-amlcompute-demo'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train.py', project_folder)

'./train-on-amlcompute-demo/train.py'

### Create environment

Create Docker based environment with scikit-learn installed.

In [6]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment("myenv")

myenv.docker.enabled = True
myenv.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

### Provision as a persistent compute target (Basic)

You can provision a persistent AmlCompute resource by simply defining two parameters thanks to smart defaults. By default it autoscales from 0 nodes and provisions dedicated VMs to run your job in a container. This is useful when you want to continously re-use the same target, debug it between jobs or simply share the resource with other users of your workspace.

* `vm_size`: VM family of the nodes provisioned by AmlCompute. Simply choose from the supported_vmsizes() above
* `max_nodes`: Maximum nodes to autoscale to while running a job on AmlCompute

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
gpu_cluster_name = "test-cluster-sn"

# Verify that cluster does not exist already
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC12',
                                                           max_nodes=4)
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

gpu_cluster.wait_for_completion(show_output=True)

### Configure & Run

Note: if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

%%time
# Shows output of the run on stdout.
run.wait_for_completion(show_output=True)

run.get_metrics()

### Provision as a persistent compute target (Advanced)

You can also specify additional properties or change defaults while provisioning AmlCompute using a more advanced configuration. This is useful when you want a dedicated cluster of 4 nodes (for example you can set the min_nodes and max_nodes to 4), or want the compute to be within an existing VNet in your subscription.

In addition to `vm_size` and `max_nodes`, you can specify:
* `min_nodes`: Minimum nodes (default 0 nodes) to downscale to while running a job on AmlCompute
* `vm_priority`: Choose between 'dedicated' (default) and 'lowpriority' VMs when provisioning AmlCompute. Low Priority VMs use Azure's excess capacity and are thus cheaper but risk your run being pre-empted
* `idle_seconds_before_scaledown`: Idle time (default 120 seconds) to wait after run completion before auto-scaling to min_nodes
* `vnet_resourcegroup_name`: Resource group of the **existing** VNet within which AmlCompute should be provisioned
* `vnet_name`: Name of VNet
* `subnet_name`: Name of SubNet within the VNet
* `admin_username`: Name of Admin user account which will be created on all the nodes of the cluster
* `admin_user_password`: Password that you want to set for the user account above
* `admin_user_ssh_key`: SSH Key for the user account above. You can specify either a password or an SSH key or both
* `remote_login_port_public_access`: Flag to enable or disable the public SSH port. If you dont specify, AmlCompute will smartly close the port when deploying inside a VNet

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
gpu_cluster_name = "test-cluster-prv"

# Verify that cluster does not exist already
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC12',
                                                           vm_priority='lowpriority',
                                                           min_nodes=2,
                                                           max_nodes=4,
                                                           idle_seconds_before_scaledown='300')
#     ,
#                                                            vnet_resourcegroup_name='<my-resource-group>',
#                                                            vnet_name='<my-vnet-name>',
#                                                            subnet_name='<my-subnet-name>',
#                                                            admin_username='<my-username>',
#                                                            admin_user_password='<my-password>',
#                                                            admin_user_ssh_key='<my-sshkey>',
#                                                            remote_login_port_public_access='enabled')
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

gpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
from azureml.core import ScriptRunConfig
#from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.runconfig import DEFAULT_GPU_IMAGE

src = ScriptRunConfig(source_directory=project_folder, script='train.py')

# Set compute target to the one created in previous step
src.run_config.target = gpu_cluster.name

# Set environment
src.run_config.environment = myenv
 
run = experiment.submit(config=src)
run

Experiment,Id,Type,Status,Details Page,Docs Page
train-on-amlcompute-01,train-on-amlcompute-01_1586353436_af6e574a,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


### Configure & Run

# Set compute target to the one created in previous step
src.run_config.target = gpu_cluster.name
 
run = experiment.submit(config=src)
run

In [ ]:
%%time
# Shows output of the run on stdout.
run.wait_for_completion(show_output=True)

In [ ]:
run.get_metrics()

## Additional operations to perform on AmlCompute

You can perform more operations on AmlCompute such as updating the node counts or deleting the compute. 

In [21]:
#get_status () gets the latest status of the AmlCompute target
gpu_cluster.get_status().serialize()


{'currentNodeCount': 2,
 'targetNodeCount': 2,
 'nodeStateCounts': {'preparingNodeCount': 0,
  'runningNodeCount': 0,
  'idleNodeCount': 2,
  'unusableNodeCount': 0,
  'leavingNodeCount': 0,
  'preemptedNodeCount': 0},
 'allocationState': 'Steady',
 'allocationStateTransitionTime': '2020-04-08T12:17:29.568000+00:00',
 'errors': None,
 'creationTime': '2020-04-08T12:15:43.138896+00:00',
 'modifiedTime': '2020-04-08T12:15:59.104313+00:00',
 'provisioningState': 'Succeeded',
 'provisioningStateTransitionTime': None,
 'scaleSettings': {'minNodeCount': 2,
  'maxNodeCount': 4,
  'nodeIdleTimeBeforeScaleDown': 'PT300S'},
 'vmPriority': 'LowPriority',
 'vmSize': 'STANDARD_NC12'}

In [22]:
#list_nodes () gets the list of nodes on the cluster with status, IP and associated run
gpu_cluster.list_nodes()


[{'nodeId': 'tvmps_29c41689dceec9802375cccdbdb93432f9cf27d008aa2b0bb9e63150acbd5459_p',
  'port': 50000,
  'publicIpAddress': '13.66.85.244',
  'privateIpAddress': '10.0.0.4',
  'nodeState': 'idle'},
 {'nodeId': 'tvmps_c0a9db6c2fd2759e73abf907799da2b55441625f045da265b0615c1f7751abc9_p',
  'port': 50001,
  'publicIpAddress': '13.66.85.244',
  'privateIpAddress': '10.0.0.5',
  'nodeState': 'idle'}]

In [23]:
#Update () takes in the min_nodes, max_nodes and idle_seconds_before_scaledown and updates the AmlCompute target
#cpu_cluster.update(min_nodes=1)
#cpu_cluster.update(max_nodes=10)
gpu_cluster.update(idle_seconds_before_scaledown=100)
#cpu_cluster.update(min_nodes=2, max_nodes=4, idle_seconds_before_scaledown=600)

In [ ]:
#Delete () is used to deprovision and delete the AmlCompute target. Useful if you want to re-use the compute name 
#'cpu-cluster' in this case but use a different VM family for instance.

#cpu_cluster.delete()

## Success!
Great, you are ready to move on to the remaining notebooks.